In [22]:
import requests
import json
import xml.etree.ElementTree as ET

def get_osm_map_bounds(osm_map_dir):
    bounds = ET.parse(osm_map_dir).getroot().find("bounds")
    return dict(bounds.attrib.items())

def get_region_issues(osm_map_dir, city_key, labels):
    coords = get_osm_map_bounds(osm_map_dir)
    print(coords)
    resp = requests.get(
        f"https://sidewalk-{city_key}.cs.washington.edu/v3/api/labelClusters",
        params={ "bbox": f"{coords["minlon"]},{coords["minlat"]},{coords["maxlon"]},{coords["maxlat"]}", "labelType": ",".join(labels) },
        timeout=20
    )
    if resp.status_code == 200:
        data = resp.json()
        with open("labels.json", "w") as f:
            json.dump(data, f, indent=4)

get_region_issues("./test_data/small_map.osm", "sea", ["NoCurbRamp", "SurfaceProblem", "NoSidewalk"])

{'minlat': '47.6842390', 'minlon': '-122.3574660', 'maxlat': '47.6878060', 'maxlon': '-122.3485950'}
